## save_correction_models_to_DB
* save metadata correction models in postgres database
* create table model_coeffs that has metadata correction model coefficients
* tables col_means and col_stds store the means and standard deviations of predictors so predicted scaled sales can be transformed back to original scale.

In [3]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

from sqlalchemy import create_engine
import psycopg2 
import io

import pickle

In [71]:
dept_list =[i for i in range(2,82)]
model_coeffs  = pd.read_pickle('data/model_coeffs_dept_1.pkl')
col_means  = pd.read_pickle('data/col_means_dept_1.pkl')
col_stds  = pd.read_pickle('data/col_stds_dept_1.pkl')


In [72]:
error_list = []

for i in dept_list:
    try:
        df =  pd.read_pickle('data/model_coeffs_dept_'+str(i)+'.pkl')
        model_coeffs = pd.concat([model_coeffs,df])
        df =  pd.read_pickle('data/col_stds_dept_'+str(i)+'.pkl')
        col_stds = pd.concat([col_stds,df])  
        df =  pd.read_pickle('data/col_means_dept_'+str(i)+'.pkl')
        col_means = pd.concat([col_means,df])   
    except:
        error_list.append(i)
        continue

In [74]:
col_stds.columns

Index(['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Weekly_Sales', 'Dept',
       'Store'],
      dtype='object')

In [121]:
connection_args = {
    'host': '34.220.165.163', # current aws instance public IP
    'user': 'ubuntu',    
    'dbname': 'ubuntu',   
    'port': 5432         
}

In [122]:
connection = pg.connect(**connection_args)

In [123]:
cur = connection.cursor()

In [47]:


cur.execute("""CREATE TABLE model_coeffs (
  intercept float,
  IsHoliday float, 
  scaled_Temperature float, 
  scaled_Fuel_Price float,
  scaled_MarkDown1 float, 
  scaled_MarkDown2 float, 
  scaled_MarkDown3 float,
  scaled_MarkDown4 float, 
  scaled_MarkDown5 float, 
  scaled_CPI float,
  scaled_Unemployment float, 
  IsNotHoliday float, 
  predicted_scaled_sales float, 
  Dept int,
  Store int
  
);""")

In [48]:
output = io.StringIO()
model_coeffs.to_csv(output, sep='\t', header=False, index=False)
output.seek(0)
contents = output.getvalue()
cur.copy_from(output, 'model_coeffs', null="") #null values become ''   
connection.commit()

In [78]:
col_stds.columns

Index(['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Weekly_Sales', 'Dept',
       'Store'],
      dtype='object')

In [79]:
cur.execute("""CREATE TABLE col_stds (
    Temperature float, 
    Fuel_Price float, 
    MarkDown1 float, 
    MarkDown2 float, 
    MarkDown3 float,
    MarkDown4 float, 
    MarkDown5 float, 
    CPI float, 
    Unemployment float, 
    Weekly_Sales float, 
    Dept int,
    Store int
  
);""")

In [80]:
output = io.StringIO()
col_stds.to_csv(output, sep='\t', header=False, index=False)
output.seek(0)
contents = output.getvalue()
cur.copy_from(output, 'col_stds', null="") #null values become ''   
connection.commit()

In [81]:
cur.execute("""CREATE TABLE col_means (
    Temperature float, 
    Fuel_Price float, 
    MarkDown1 float, 
    MarkDown2 float, 
    MarkDown3 float,
    MarkDown4 float, 
    MarkDown5 float, 
    CPI float, 
    Unemployment float, 
    Weekly_Sales float, 
    Dept int,
    Store int
  
);""")

In [82]:
output = io.StringIO()
col_means.to_csv(output, sep='\t', header=False, index=False)
output.seek(0)
contents = output.getvalue()
cur.copy_from(output, 'col_means', null="") #null values become ''   
connection.commit()

In [54]:
df = pd.read_sql_query('select * from model_coeffs where store=1 and dept=1',con=connection)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 15 columns):
intercept                 1 non-null float64
isholiday                 1 non-null float64
scaled_temperature        1 non-null float64
scaled_fuel_price         1 non-null float64
scaled_markdown1          1 non-null float64
scaled_markdown2          1 non-null float64
scaled_markdown3          1 non-null float64
scaled_markdown4          1 non-null float64
scaled_markdown5          1 non-null float64
scaled_cpi                1 non-null float64
scaled_unemployment       1 non-null float64
isnotholiday              1 non-null float64
predicted_scaled_sales    1 non-null float64
dept                      1 non-null int64
store                     1 non-null int64
dtypes: float64(13), int64(2)
memory usage: 200.0 bytes
